# Prepare Inputs For Hapi

To start using Hapi, the meteorological Input data (Precipitation, potential Evapotranspiration and Temperature) have to match the DEM in the number of rows and columns and for this purpose this code takes the inputs and (Raster form) and prepare them to be ready to run Hapi

### - import Libraries

In [ ]:
#library
import numpy as np
# Hapi Modules
from Hapi import Raster
from Hapi import Inputs

import gdal

This Note book prepares downloaded raster data to have the same alignment and
nodatavalue from a GIS raster (DEM, flow accumulation, flow direction raster)
and return a folder with the output rasters with a name “New_Rasters”
- all the meteorological inputs has to match the DEM raster

### - Path to the DEM


In [ ]:
dem_path="Data/GIS/4000/acc4000.tif"

outputpath="Data/meteodata_prepared/"

You need to create three folders inside the outputpath defined the the previous cell
- /outputpath
    /prec
    /evap
    /temp

### Meteorological Input folders
each folder contain the rasters you want to prepare for running Hapi

In [ ]:
# prec
prec_in_path="Data/meteodata/4000/complete_dataset/prec/"

### prepareInputs 
this function takes the DEM as a template (rows and columns) and takes the folder where any raster data
you want to match with exists

- First it creates a folder in your temp folder to allign (match the number od rows and columns) in all the rasters with the DEM 
- Second it crops all the rasters to the same extent of the DEM and creates a folder with the given string in the third argument (if already exist it will overwrite)

In [ ]:
# Run the code 
Inputs.prepareInputs(dem_path,prec_in_path,outputpath+"prec")

In [ ]:
# evap
evap_in_path="Data/meteodata/4000/complete_dataset/evap/"
Inputs.prepareInputs(dem_path,evap_in_path,outputpath+"evap")

In [ ]:
# temp
temp_in_path="Data/meteodata/4000/complete_dataset/temp/"
Inputs.prepareInputs(dem_path,temp_in_path,outputpath+"temp")

### Map Algebra

in case you want to manipulate the value in all the rasters of one of the inputs 
for example evapotranspiration values in rasters downloaded from ECMWF are negative
(the sign is indication of the direction of the flux so evaporation is negativve and condensation is positive
"How can evaporation have both positive and negative values? 
Evaporation is normally negative due to the convention for fluxes.
The meteorological convention for all vertical fluxes is that downwards is positive.
Positive evaporation represents condensation".
Link: https://confluence.ecmwf.int/pages/viewpage.action?pageId=111155327)
and to change it to positive in all rasters 
or 
if you want to operate any kind of function in all input rasters that are in the same
folder FolderCalculator function can do this task

In [ ]:
# define your function
# this function is going to take the absolute value of the values in the raster
# through MapAlgebra function then save the new raster to a given path with the same names
def function(args):
    # argument a list of two components
    # first argument is the raster object [gdal object]
    A = args[0]
    # second argument is the path to save the resulted raster
    path = args[1]
    func=np.abs
    # first function 
    B=Raster.MapAlgebra(A,func)
    Raster.SaveRaster(B,path)

In [ ]:
# the inputs will be the evaporation rasters that we have just alligned
evap_out_path="Data/meteodata_prepared/evap/"
folder_path = evap_out_path
new_folder_path="data/meteodata_prepared/new_evap/"

In [ ]:
Raster.FolderCalculator(folder_path,new_folder_path,function)

### Align any raster (MatchRasterAlignment function)

You need to read the source raster and the raster you want to match it to the source (match alignment coordinate system and 
cell size) you need to read both rasters with gdal and path them to MatchRasterAlignment function

In [ ]:
soil_path="Data/GIS/soil/4000/soil_raster.tif"
DEM=gdal.Open(dem_path)
soil=gdal.Open(soil_path)
#dem_A=DEM.ReadAsArray()
#soil_A=soil.ReadAsArray()

In [ ]:
dem_A=DEM.ReadAsArray()
print(np.shape(dem_A))

In [ ]:
soil_A=soil.ReadAsArray()
print(np.shape(soil_A))

In [ ]:
# MatchRasterAlignment
# MatchRasterAlignment function returns a gdal object 
aligned_soil=Raster.MatchRasterAlignment(DEM,soil)

In [ ]:
# to check alignment of DEM raster compared to aligned_soil_A raster
aligned_soil_A=aligned_soil.ReadAsArray()

In [ ]:
print(np.shape(aligned_soil_A))

In [ ]:
# nodatavalue is still different and some cells are no data value in the soil type raster but it is not in the dem raster
# to match use Match MatchNoDataValue
# match
dst_Aligned_M=Raster.MatchNoDataValue(DEM,aligned_soil)
dst_Aligned_M_A=dst_Aligned_M.ReadAsArray()

In [ ]:
# save the raster
Raster.SaveRaster(dst_Aligned_M,"Data/GIS/soil/4000/soil_type.tif")